In [ ]:
import os
import json
import requests
import progressbar
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [ ]:
webpage = requests.get('https://www.premierleague.com/stats/top/clubs/wins?se=79')
soup = BeautifulSoup(webpage.text, 'html.parser')

if not os.path.exists('files'):
    os.makedirs('files/stats')
    os.makedirs('files/results')

In [ ]:
def attributes(links):
    return [link[link.rfind('/')+1:] for link in links]

def uniques(links):
    l = []
    for link in links:
        if link not in l:
            l.append(link)
    return l

top = [link['href'] for link in soup.select('a.topStatsLink')]
more = [link['href'] for link in soup.select('nav.moreStatsMenu a')]
links = uniques(attributes(more) + attributes(top))

In [ ]:
dates = {'2006-2007':15, '2007-2008':16, '2008-2009':17, '2009-2010':18,
         '2010-2011':19, '2011-2012':20, '2012-2013':21, '2013-2014':22,
         '2014-2015':27, '2015-2016':42, '2016-2017':54, '2017-2018':79,
         '2018-2019':210, '2019-2020':274, '2020:2021':363, '2021-2022':418,
         '2022-2023':489, '2023-2024':578}

In [ ]:
for date in dates.keys():

    df = pd.DataFrame()
    bar = progressbar.ProgressBar(maxval=len(links), widgets=[date + '\t', progressbar.Bar('-', '[', ']'), ' ', progressbar.Percentage()])
    bar.start()
    for i, attribute in zip(range(len(links)), links):

        # setup
        api = 'https://footballapi.pulselive.com/football/stats/ranked/teams/' + attribute
        headers = {'Origin': 'https://www.premierleague.com'}
        params = {'page': '0', 'pageSize': '20', 'compSeasons': dates[date], 'comps': '1', 'altIds': 'true'}

        # request
        response = requests.get(api, params=params, headers=headers)
        data = json.loads(response.text)

        # parse
        teams = []; values = [];
        for team in data['stats']['content']:
            teams.append(team['owner']['name'])
            values.append(team['value'])
        series = pd.Series(values, teams, float, attribute)
        if df.index.empty:
            df = pd.DataFrame(series)
        else:
            df = df.join(series)

        # progress
        bar.update(i+1)

    bar.finish()
    df.dropna(axis=1, how='all', inplace=True)
    df.fillna(0, inplace=True)
    df.to_csv('files/stats/' + date + '.csv')

2006-2007	[--------------------------------------------------------------] 100%
2007-2008	[--------------------------------------------------------------] 100%
2008-2009	[--------------------------------------------------------------] 100%
2009-2010	[--------------------------------------------------------------] 100%
2010-2011	[--------------------------------------------------------------] 100%
2011-2012	[--------------------------------------------------------------] 100%
2012-2013	[--------------------------------------------------------------] 100%
2013-2014	[--------------------------------------------------------------] 100%
2014-2015	[--------------------------------------------------------------] 100%
2015-2016	[--------------------------------------------------------------] 100%
2016-2017	[--------------------------------------------------------------] 100%
2017-2018	[--------------------------------------------------------------] 100%
2018-2019	[-----------------------------

In [ ]:
datasets = []
for date in dates.keys():
    dataset = pd.read_csv('files/stats/' + date + '.csv', index_col=0)
    datasets.append(dataset)
    print(date + '\t' + str(len(dataset.columns)))

2006-2007	19
2007-2008	20
2008-2009	20
2009-2010	20
2010-2011	21
2011-2012	21
2012-2013	21
2013-2014	21
2014-2015	21
2015-2016	21
2016-2017	21
2017-2018	21
2018-2019	21
2019-2020	21
2020:2021	21
2021-2022	21
2022-2023	21
2023-2024	21


In [ ]:
mismatches = []

for i in range(4,12):
    for j in range(i+1,12):
        if datasets[i].columns.tolist() != datasets[j].columns.tolist():
            mismatches.append((i,j))

if len(mismatches) == 0:
    print('Valid Set')
else:
    print('Invalid Set')

Valid Set


In [ ]:
def get_team_ids(date):
    # setup
    api = 'https://footballapi.pulselive.com/football/compseasons/' + str(dates[date]) + '/teams'
    headers = {'Origin': 'https://www.premierleague.com'}

    # request
    response = requests.get(api, headers=headers)
    teams = json.loads(response.text)

    # parse
    team_ids = []
    for team in teams:
        team_ids.append(int(team['id']))
    team_ids = ','.join(map(str, team_ids))

    return team_ids

def get_results(date, team_ids):
    # setup
    api = 'https://footballapi.pulselive.com/football/fixtures'
    headers = {'Origin': 'https://www.premierleague.com'}
    params = {'comps':'1', 'compSeasons':dates[date], 'teams':team_ids, 'page':'0', 'pageSize':'380', 'sort':'asc', 'statuses':'C', 'altIds':'true'}

    # request
    response = requests.get(api, params=params, headers=headers)
    results = json.loads(response.text)

    # parse
    df = pd.DataFrame(columns=['home_team', 'away_team', 'home_goals', 'away_goals', 'result'])
    for result in results['content']:
        row = []
        row.append(result['teams'][0]['team']['name'])
        row.append(result['teams'][1]['team']['name'])
        row.append(result['teams'][0]['score'])
        row.append(result['teams'][1]['score'])
        row.append(result['outcome'])
        row = pd.Series(row, index=df.columns)
        df = df.append(row, ignore_index=True)

    return df

bar = progressbar.ProgressBar(maxval=len(dates), widgets=['', '\t', progressbar.Bar('-', '[', ']'), ' ', progressbar.Percentage()])
bar.start()
for i, date in zip(range(len(dates)), dates.keys()):
    bar.widgets[0] = date
    team_ids = get_team_ids(date)
    results = get_results(date, team_ids)
    bar.update(i+1)
    results.to_csv('files/results/' + date + '.csv')
bar.finish()

Streaming output truncated to the last 5000 lines.
<ipython-input-9-7c5af9444164>:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
<ipython-input-9-7c5af9444164>:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
<ipython-input-9-7c5af9444164>:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
<ipython-input-9-7c5af9444164>:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
<ipython-input-9-7c5af9444164>:38: FutureWarning: The frame.append method is deprecated and will be removed from 

In [ ]:
files = ['2006-2007.csv', '2007-2008.csv', '2008-2009.csv', '2009-2010.csv', '2010-2011.csv', '2011-2012.csv', '2012-2013.csv', '2013-2014.csv', '2014-2015.csv', '2015-2016.csv', '2016-2017.csv', '2017-2018.csv']

stats_df = pd.DataFrame()
results_df = pd.DataFrame()
for name in files:

    # Stats
    f = 'files/stats/' + name
    stats_series = pd.Series([name[:-4]]*20, name='season')
    stats_season = pd.concat([pd.read_csv(f, index_col=False), stats_series], axis=1)
    columns = stats_season.columns.tolist(); columns[0] = 'team'; stats_season.columns = columns
    if stats_df.empty:
        stats_df = stats_season
    else:
        stats_df = pd.concat([stats_df, stats_season])

    # Results
    f = 'files/results/' + name
    results_series = pd.Series([name[:-4]]*380, name='season')
    results_season = pd.concat([pd.read_csv(f), results_series], axis=1)
    if results_df.empty:
        results_df = results_season
    else:
        results_df = pd.concat([results_df, results_season])

stats_df = stats_df[stats_season.columns.tolist()]
stats_df.to_csv('files/stats/stats.csv', index=False)

results_df.drop(results_df.columns[0], axis=1, inplace=True)
results_df.to_csv('files/results/results.csv', index=False)